In [1]:
elements = 8
POPULATION_SIZE = 15
MIXING_NUMBER = 2
MUTATION_RATE = 0.05

In [2]:
def fitness_score(seq):
    score = 0

    for row in range(elements):
        col = seq[row]

        for other_row in range(elements):

            
            if other_row == row:
                continue
            if seq[other_row] == col:
                continue
            if other_row + seq[other_row] == row + col:
                continue
            if other_row - seq[other_row] == row - col:
                continue
            
            score += 1

    
    return score/2

In [4]:
import random
from scipy import special as sc

def selection(population):
    parents = []

    for ind in population:
        
        if random.randrange(sc.comb(elements, 2)*2) < fitness_score(ind):
            parents.append(ind)


    return parents

ModuleNotFoundError: No module named 'scipy'

In [ ]:
import itertools

def crossover(parents):

    
    cross_points = random.sample(range(elements), MIXING_NUMBER - 1)
    offsprings = []

    
    permutations = list(itertools.permutations(parents, MIXING_NUMBER))

    for perm in permutations:
        offspring = []

        
        start_pt = 0

        for parent_idx, cross_point in enumerate(cross_points):

            
            parent_part = perm[parent_idx][start_pt:cross_point]
            offspring.append(parent_part)

            
            start_pt = cross_point

        
        last_parent = perm[-1]
        parent_part = last_parent[cross_point:]
        offspring.append(parent_part)

        
        offsprings.append(list(itertools.chain(*offspring)))

    return offsprings

In [ ]:
def mutate(seq):
    for row in range(len(seq)):
        if random.random() < MUTATION_RATE:
            seq[row] = random.randrange(elements)
 
    return seq

In [ ]:
def print_found_goal(population, to_print=True):
    for ind in population:
        score = fitness_score(ind)
        if to_print:
            print(f'{ind}. Score: {score}')
        if score == sc.comb(elements, 2):
            if to_print:
                print('Solution found')
            return True

    if to_print:
        print('Solution not found')
    return False

In [ ]:
def evolution(population):
    
    parents = selection(population)

    
    offsprings = crossover(parents)

    
    offsprings = list(map(mutate, offsprings))

    
    new_gen = offsprings

    for ind in population:
        new_gen.append(ind)

    new_gen = sorted(new_gen, key=lambda ind: fitness_score(ind), reverse=True)[:POPULATION_SIZE]

    return new_gen

In [ ]:
def generate_population():
    population = []

    for individual in range(POPULATION_SIZE):
        new = [random.randrange(elements) for idx in range(elements)]
        population.append(new)

    return population

In [ ]:
generation = 0


population = generate_population()

while not print_found_goal(population):
    print(f'Generation: {generation}')
    print_found_goal(population)
    population = evolution(population)
    generation += 1

In [ ]:
import numpy as np


total_sum = []
for i in range(10000):
    population = generate_population()
    for score in list(map(fitness_score, population)):
        total_sum.append(score)
print(f'Mean: {np.mean(total_sum)}')
print(f'St. dev: {np.std(total_sum)}')

In [ ]:
gens = []
for run in range(200):
    generation = 0
    population = generate_population()
    print(f'Run: {run}')
    while not print_found_goal(population, to_print=False):
        population = evolution(population)
        generation += 1

    gens.append(generation)

print(f'Mean: {np.mean(gens)}')
print(f'St. dev: {np.std(gens)}')

In [ ]:
print(f'Min: {min(gens)}')
print(f'Max: {max(gens)}')

In [ ]:
print('Stats from E1-E3')
print(f'Min: {min(gens[50:150])}')
print(f'Max: {max(gens[50:150])}')
print(f'Mean: {np.mean(gens[50:150])}')
print(f'St. dev: {np.std(gens[50:150])}')

In [ ]:
def reject_outliers(data, m = 2.):
    d = np.abs(data - np.median(data))
    mdev = np.median(d)
    s = d/mdev if mdev else 0.
    return data[s<m]

no_outliers = reject_outliers(np.array(gens))

print('Removed Outliers')
print(f'Min: {min(no_outliers)}')
print(f'Max: {max(no_outliers)}')
print(f'Mean: {np.mean(no_outliers)}')
print(f'St. dev: {np.std(no_outliers)}')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

y = no_outliers

sns.set()
plt.hist(y, bins=np.arange(0, max(no_outliers), 20))
plt.xlabel('Generations to reach solution', fontsize=16)
plt.ylabel('Count', fontsize=20)
plt.title('Distribution of Number of Generations to reach solution with a Genetic Algorithm (Outliers removed)', fontsize=33)
plt.rcParams["figure.figsize"] = (20,10)
plt.xticks(size = 15)
plt.yticks(size = 15)
plt.axvline(x=np.mean(no_outliers), label='Mean', ls='--')
plt.show()